In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data.dataloader import DataLoader
import numpy as np
from sklearn.utils import shuffle # for shuffling
import os
import cv2
import random
import torch.nn as nn
import torch.optim as optim

In [2]:
from tqdm import tqdm

In [3]:
import gc

In [4]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip

--2024-04-01 16:50:37--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.207, 108.177.96.207, 108.177.119.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: 'nature_12K.zip'

nature_12K.zip      100%[===================>]   3.55G  42.2MB/s    in 88s     

2024-04-01 16:52:05 (41.4 MB/s) - 'nature_12K.zip' saved [3816687935/3816687935]



In [5]:
!rm nature_12K.zip

In [6]:
dtype = torch.float
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
print(device)

cuda


In [8]:
resize_width = 500
resize_height= 500

In [9]:
# Defining your transformations
transform = transforms.Compose([
    transforms.Resize((resize_width, resize_height)),  # Resize to 256x256
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])

# Dataset
TrainDataset = datasets.ImageFolder(root='inaturalist_12K/train', transform=transform)
# DataLoader with shuffling
TrainData_loader = DataLoader(TrainDataset,shuffle=True,num_workers=2,batch_size=64,pin_memory=True)

# same for validation
ValidationDataset = datasets.ImageFolder(root='inaturalist_12K/val', transform=transform)
# DataLoader with shuffling
ValidationData_loader = DataLoader(ValidationDataset, batch_size=64)

In [10]:
type(TrainData_loader)

torch.utils.data.dataloader.DataLoader

In [11]:
for batch in TrainData_loader:
    for tensor in batch:
        print(tensor.device)
        break
    break

cpu


In [12]:
images , labels = next(iter(TrainData_loader))
images.size()

torch.Size([64, 3, 500, 500])

In [13]:
def getDimOfLastConv(num_filters, filter_sizes, use_batch_norm=False):
    layers = []
    
    # Initial convolution layer
    layers.append(nn.Conv2d(3, num_filters[0], kernel_size=filter_sizes[0], stride=1, padding=0))
    layers.append(nn.ReLU())
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    if use_batch_norm:
        layers.append(nn.BatchNorm2d(num_filters[0]))
    
    # Subsequent convolution layers
    for i in range(1, len(num_filters)):
        layers.append(nn.Conv2d(num_filters[i-1], num_filters[i], kernel_size=filter_sizes[i], stride=1, padding=0))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        if use_batch_norm:
            layers.append(nn.BatchNorm2d(num_filters[i]))
    
    conv_stack = nn.Sequential(*layers)
    
    image_tensor = torch.zeros([1, 3, resize_width, resize_height])
    x = conv_stack(image_tensor)
    flat = nn.Flatten()
    x = flat(x)
    return x.shape[-1]

In [14]:
class SimpleCNN(nn.Module):
  def __init__(self, num_filters, filter_sizes, activation_fn, num_neurons_dense, use_batch_norm = True):
    super(SimpleCNN, self).__init__()

    layers = []
    
    # Initial convolution layer
    layers.append(nn.Conv2d(3, num_filters[0], kernel_size=filter_sizes[0], stride=1, padding=0))
    layers.append(activation_fn)
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    if use_batch_norm:
        layers.append(nn.BatchNorm2d(num_filters[0]))
    
    # Subsequent convolution layers
    for i in range(1, len(num_filters)):
        layers.append(nn.Conv2d(num_filters[i-1], num_filters[i], kernel_size=filter_sizes[i], stride=1, padding=0))
        layers.append(activation_fn)
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        if use_batch_norm:
            layers.append(nn.BatchNorm2d(num_filters[i]))
    
    self.conv_stack = nn.Sequential(*layers)

    flattenNodes = getDimOfLastConv(num_filters, filter_sizes, use_batch_norm)
    self.flatten = nn.Flatten()
    self.dense = nn.Linear(flattenNodes, num_neurons_dense)
    self.output = nn.Linear(num_neurons_dense, 10)

  def forward(self, x):
    x = self.conv_stack(x)  # Pass input through conv_stack
    x = self.flatten(x)  # Flatten the output of conv layers
    x = self.dense(x) # passing through dense layer
    x = self.output(x)
    return x


In [ ]:
def train_model(learning_rate, num_filters, filter_sizes, activation_fn, num_neurons_dense, weight_decay):
    
num_epochs = 10
learning_rate = 0.001
num_filters = [32, 32, 32, 32, 32]  
filter_sizes = [3, 3, 3, 3, 3]
activation_fn = nn.ReLU()
num_neurons_dense = 400

model = SimpleCNN(num_filters=num_filters, filter_sizes=filter_sizes, 
                  activation_fn=activation_fn, num_neurons_dense=num_neurons_dense)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.nn.DataParallel(model,device_ids = [0,1]).to(device)

def train(model, criterion, optimizer, num_epochs, train_loader, val_loader):
    for epoch in range(num_epochs):
        model.train()
#         for inputs, labels in train_loader:
        for ind, (inputs, labels) in enumerate(tqdm(train_loader, desc='Training Progress')):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        find_accuracy(model, criterion, train_loader,"train")
        find_accuracy(model, criterion, val_loader, "validation")

def find_accuracy(model, criterion, dataLoader, dataName):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in dataLoader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    print(f'{dataName} Loss: {val_loss/len(dataLoader)}, '
          f'{dataName} Accuracy: {100*correct/total}%\n')

train(model, criterion, optimizer, num_epochs, TrainData_loader, ValidationData_loader)


In [16]:
model.cpu()
del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:

sweep_config = {
    'method': 'bayes',
    'name' : 'sweep cross entropy',
    'metric': {
      'name': 'v_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'num_filters': {
          'values': [[32,32,32,32,32],[32,64,64,128,128],[128,128,64,64,32],[32,64,128,256,512]]
        },
        'filter_sizes': {
          'values': [[3,3,3,3,3], [5,5,5,5,5], [3,5,3,5,3]]
        },
        'weight_decay': {
            'values':[0, 0.0005, 0.5]
        },
        'learning_rate': {
            'values':[1e-3,1e-4]
        },
        'weight_decay': {
            'values': [0, 0.0005, 0.005]
        },
        'dropout': {
            'values': [0, 0.2, 0.4]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'activation': {
            'values': ['relu', 'elu', 'selu']
        },
        'optimiser': {
            'values': ['nadam', 'adam', 'mgd']
        }
        'batch_norm':{
            'values': ['true','false']
        },
        'data_augment': {
            'values': ['true','false']
        },
        'batch_size': {
            'values': [32, 64]
        },
        'dense_layer':{
            'values': [64, 128, 256, 512]
        }
    }
}